# Battle of Neighborhoods Week 1

## 1 Introduction Section- Description of the problem and the background

I am a PhD scholar residing in Kolkata, India. I currently live in Newtown area of Kolkata and I enjoy many amenities and venues in the area, such as various restaurants, cafes, food shops and entertainment. I have shifted to Mumbai, India for working purpose. The key question is: How can I find a convenient and enjoyable place in Navi Mumbai similar to that in Kolkata? In order to make a comparison and evaluation of the rental options in Navi Mumbai, I must set some basis, therefore the apartment in Navi Mumbai must meet the following demands:
•	Apartment must be 2 or 3 bedrooms
•	Desired location is near a railway station in the Navi Mumbai area and within 2 km
•	Price of rent not exceed INR 15,000 per month
•	Top amenities in the selected neighborhood shall be similar to current residence
•	Desirable to have venues such as market, restaurants, and wine stores


## Business Problem


The challenge is to find a suitable apartment for rent in Navi Mumbai that complies with the demands on location, price ,venues and so on.

## Data Section

The data will be used as follows:
•	Use Foursquare and geopy data to map top 10 venues for all Navi Mumbai neighborhoods and clustered in groups ( as per Course LAB)
•	Use Foursquare and geopy data to map the location of railway stations , separately and on top of the above clustered map in order to be able to identify the venues and amenities near each railway station
•	Use Foursquare and geopy data to map the location of rental places, in some form, linked to the railway stations.
•	create a map that depicts, for instance, the average rental price per square ft, around a radius of 2 km around each railway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per railway station.
•	Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
•	Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies.


The processing of these DATA will allow to answer the key questions to make a decision:
•	what is the cost of rent (per square ft) around 2 km radius from each railway station?
•	what is the area of Navi Mumbai with best rental pricing that meets criteria established?
•	What is the distance from work place (Vasi, Navi Mumbai) and the tentative future home?
•	What are the venues of the two best places to live? How the prices compare?
•	How venues distribute among Navi Mumbai neighborhoods and around railway stations?
•	Are there tradeoffs between size and price and location?
•	Any other interesting statistical data findings of the real estate and overall data.


Importing required libraries

In [25]:
import numpy as np
import time
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
import requests
from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim #
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: ...working... done

# All requested packages already installed.



In [2]:
# New Town, Kolkata
address = 'New Town, Kolkata'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Town Kolkata are {}, {}.'.format(latitude, longitude))

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


GeocoderTimedOut: Service timed out

In [48]:
neighborhood_latitude=22.5724
neighborhood_longitude=88.4798

In [49]:
CLIENT_ID = 'VGVN1IKTDQGYGE4CJQ1OFGMNA44HJ54QI4OWT5WGSRGNKAG1'
CLIENT_SECRET = 'ZHVB0VEIGTDLW2FZJGVQRDU33BBBRWSYUP0VIBZ2BLROLTEF'
VERSION = '20191001'

In [50]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=VGVN1IKTDQGYGE4CJQ1OFGMNA44HJ54QI4OWT5WGSRGNKAG1&client_secret=ZHVB0VEIGTDLW2FZJGVQRDU33BBBRWSYUP0VIBZ2BLROLTEF&v=20191001&ll=22.5724,88.4798&radius=500&limit=100'

In [51]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5da22adc531593002c653044'},
 'response': {'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 22.576900004500004,
    'lng': 88.48466422534415},
   'sw': {'lat': 22.567899995499992, 'lng': 88.47493577465585}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50a9f08de4b02dc98e167a6b',
       'name': 'Flavours',
       'location': {'address': 'Unitech Infospace',
        'lat': 22.575677201583147,
        'lng': 88.48236322402954,
        'labeledLatLngs': [{'label': 'display',
          'lat': 22.575677201583147,
          'lng': 88.48236322402954}],
        'distance': 450,
        'postalCode': '700156',
  

In [52]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [59]:
venues = results['response']['groups'][0]['items']
NewTown_Kolkata_venues = json_normalize(venues)

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
NewTown_Kolkata_venues =NewTown_Kolkata_venues.loc[:, filtered_columns]

# filter the category for each row
NewTown_Kolkata_venues['venue.categories'] = NewTown_Kolkata_venues.apply(get_category_type, axis=1)

# clean columns
NewTown_Kolkata_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

NewTown_Kolkata_venues.head(5)

,name,categories,lat,lng
0,Flavours,Indian Restaurant,22.575677,88.482363
1,Archies,Gift Shop,22.574307,88.479388
2,Tata Consultancy Services,IT Services,22.575630,88.482104
3,Cognizant Technology Solutions,IT Services,22.575618,88.482342
4,Accenture,Business Service,22.574783,88.483257
